In [81]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
import numpy as np

In [2]:
journal = 'composites-part-b-engineering'

In [3]:

result = requests.get("https://www.journals.elsevier.com/"+journal)
soup = BeautifulSoup(result.content)
metrics = soup.find_all("span", "tooltip")
met_names = ['CiteScore','ImpactFactor','5-year Impact Factor','Norm. Impact per Paper','Journal Rank']
data = [a.b.text for a in metrics if a.b != None]
del data[2:4]
met_dict = dict(zip(met_names,data))

In [4]:
met_dict

{'CiteScore': '11.1',
 'ImpactFactor': '6.864',
 '5-year Impact Factor': '6.313',
 'Norm. Impact per Paper': '2.619',
 'Journal Rank': '1.925'}

In [154]:
volumes = [str(i) for i in range(150,196)]
titles_str = []
for i in volumes:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    url = "https://www.sciencedirect.com/journal/composites-part-b-engineering/vol/" + i +"/suppl/C"
    r = requests.get(url, headers=headers)
    site = BeautifulSoup(r.content)
    titles = site.find_all("span", class_="js-article-title")
    for j in titles:
        titles_str.append(j.text)

In [155]:
sws = stopwords.words('english')
titles_list = [re.split(r'\W+',i) for i in titles_str]
titles_set = [set(i).difference(sws)-{''} for i in titles_list]

In [156]:
#total count
total_list = [x for _list in titles_set for x in _list]
total_set = list(set(total_list))

count = [total_list.count(i) for i in total_set]
total_dict = dict(zip(total_set,count))

In [157]:
total_dict = {k: v for k, v in sorted(total_dict.items(), key=lambda item: item[1],reverse=True)}

In [158]:
#Word matrix
total_num = dict((j,i) for i,j in enumerate(total_set))

In [159]:
word_mat = np.zeros((len(titles_set), len(total_set)))
for i,j in enumerate(titles_set):
    for k in list(j):
        word_mat[i,total_num[k]] = 1
        

In [161]:
def dist_calc(image):
    n = len(image)
    #k = len(c)
    S = np.empty((n, n))
    for i in range(n):
        d_i = np.linalg.norm(image[i, :] - image, ord=2, axis=1)
        S[i, :] = d_i
    return S
D = dist_calc(word_mat)


In [162]:
D

array([[0.        , 4.        , 4.24264069, ..., 3.74165739, 3.46410162,
        3.31662479],
       [4.        , 0.        , 5.47722558, ..., 4.89897949, 4.89897949,
        4.58257569],
       [4.24264069, 5.47722558, 0.        , ..., 5.09901951, 5.09901951,
        4.79583152],
       ...,
       [3.74165739, 4.89897949, 5.09901951, ..., 0.        , 4.69041576,
        4.35889894],
       [3.46410162, 4.89897949, 5.09901951, ..., 4.69041576, 0.        ,
        4.12310563],
       [3.31662479, 4.58257569, 4.79583152, ..., 4.35889894, 4.12310563,
        0.        ]])